In [48]:
! pip install torch torchvision
! pip install ftfy regex tqdm
! pip install git+https://github.com/openai/CLIP.git

! pip install datasets
! pip install pandas
! pip install pillow

! pip install pinecone

  Cloning https://github.com/openai/CLIP.git to /private/var/folders/v4/zjpk5vn16yz9tdzvyv8s29xh0000gn/T/pip-req-build-2jm_9cpf
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /private/var/folders/v4/zjpk5vn16yz9tdzvyv8s29xh0000gn/T/pip-req-build-2jm_9cpf
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [49]:
from datasets import load_dataset
import os
from tqdm import tqdm
from PIL import Image
import pandas as pd

# Load dataset
ds = load_dataset("Artificio/WikiArt", split='train')
ds = ds.select(range(30))
print(ds)
# Set output folder
output_dir = "data/raw/"
images_dir = os.path.join(output_dir, "images")
os.makedirs(images_dir, exist_ok=True)

# Save images and metadata
metadata = []

for i, example in enumerate(ds):
    img: Image.Image = example["image"]
    img_filename = f"{i}_{example['title'].replace('/', '_')}.jpg"
    img_path = os.path.join(images_dir, img_filename)

    # Save image
    img.save(img_path)

    # Collect metadata
    metadata.append({
        "file": img_filename,
        "artist": example["artist"],
        "style": example["style"],
        "genre": example["genre"],
        "title": example["title"],
    })

# Save metadata as CSV
pd.DataFrame(metadata).to_csv(os.path.join(output_dir, "metadata.csv"), index=False)
print(f"✅ Saved {len(metadata)} images and metadata.")

Dataset({
    features: ['title', 'artist', 'date', 'genre', 'style', 'description', 'filename', 'image', 'embeddings_pca512'],
    num_rows: 30
})
✅ Saved 30 images and metadata.


In [50]:
import clip
models_list = clip.available_models()
print(models_list)

['RN50', 'RN101', 'RN50x4', 'RN50x16', 'RN50x64', 'ViT-B/32', 'ViT-B/16', 'ViT-L/14', 'ViT-L/14@336px']


In [51]:
import os
import torch
import clip

from PIL import Image
import pandas as pd
from tqdm import tqdm
import numpy as np

# Load metadata
metadata_df = pd.read_csv("data/raw/metadata.csv")

# CLIP setup
device = "cuda" if torch.cuda.is_available() else "cpu"
clip_model = 'RN101'
model, preprocess = clip.load(clip_model, device=device)

# Store embeddings + metadata
embeddings_data = []

# Iterate through all rows in metadata
for _, row in tqdm(metadata_df.iterrows(), total=len(metadata_df)):
    filename = row["file"]
    image_path = os.path.join("data/raw/images", filename)

    # Preprocess and encode image
    image = preprocess(Image.open(image_path)).unsqueeze(0).to(device)

    with torch.no_grad():
      image_embedding = model.encode_image(image).cpu().numpy().flatten().tolist()

    # Optionally get text embedding too (e.g., from title)
    # text = clip.tokenize([row["title"]]).to(device)
    # with torch.no_grad():
    #     text_embedding = model.encode_text(text).cpu().numpy().flatten()

    # Store all in a dictionary
    entry = {
        "file": filename,
        "title": row["title"],
        "artist": row["artist"],
        "style": row["style"],
        "genre": row["genre"],
        "image_embedding": image_embedding
        # "text_embedding": text_embedding.tolist()
    }
    embeddings_data.append(entry)

# (Optional) Save to JSON or DataFrame
# import json
# with open("image_embeddings.json", "w") as f:
#     json.dump(embeddings_data, f)

# Or convert to DataFrame if you don’t need the full vector inline
df = pd.DataFrame(embeddings_data)
print("\n🔢 Sample Embedding Vector:\n", df["image_embedding"].iloc[0][:10])  # Show first 10 dims
df.to_csv("data/processed/metadata_embeddings.csv")
df

100%|██████████| 30/30 [00:03<00:00,  9.90it/s]


🔢 Sample Embedding Vector:
 [0.004297453910112381, 0.05464690178632736, -0.036458779126405716, 0.08413168787956238, -0.055270470678806305, -0.01814919151365757, 0.06191125512123108, 0.011376820504665375, 0.013581208884716034, 0.02576671540737152]


,file,title,artist,style,genre,image_embedding
0,0_Cornelia Street.jpg,Cornelia Street,John French Sloan,New Realism,cityscape,"[0.004297453910112381, 0.05464690178632736, -0..."
1,"1_O Diabo, a Paraquedista, Etc.jpg","O Diabo, a Paraquedista, Etc",Rene Bertholo,Surrealism,figurative,"[-0.01440428476780653, -0.043104857206344604, ..."
2,2_Milking the Cow .jpg,Milking the Cow,Yasuo Kuniyoshi,Expressionism,genre painting,"[0.014192033559083939, -0.016326740384101868, ..."
3,3_Self-portrait.jpg,Self-portrait,Lajos Tihanyi,Expressionism,self-portrait,"[0.033301930874586105, -0.014760628342628479, ..."
4,4_All healing.jpg,All healing,Raphael Kirchner,Art Nouveau (Modern),genre painting,"[0.0161421075463295, -0.0597282350063324, -0.1..."
5,5_A fresco by M. Nesterov from Akhali Zarzma m...,A fresco by M. Nesterov from Akhali Zarzma mon...,Mikhail Nesterov,Symbolism,religious painting,"[-0.015609091147780418, 0.01829567179083824, -..."
6,6_Winter.jpg,Winter,Isaac Levitan,Realism,landscape,"[-0.022748354822397232, 0.013287687674164772, ..."
7,7_Sketch of figures and scenes from the antiqu...,Sketch of figures and scenes from the antique age,Jacopo Bellini,Early Renaissance,sketch and study,"[0.010421626269817352, -0.0655362606048584, -0..."
8,8_The Horsemen of the Apocalypse .jpg,The Horsemen of the Apocalypse,Carlo Carra,Divisionism,religious painting,"[0.007174206897616386, 0.05179232731461525, -0..."
9,9_Arctic Seascape.jpg,Arctic Seascape,William Bradford,Romanticism,marina,"[0.03983573243021965, 0.011275490745902061, -0..."


In [ ]:
# Import the Pinecone library
from pinecone import Pinecone, ServerlessSpec
from dotenv import load_dotenv

load_dotenv()

PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
# Initialize a Pinecone client with your API key
pc = Pinecone(api_key=PINECONE_API_KEY)

# Create a dense index with integrated embedding
index_name = "frame-finder-database"
if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        dimension=512,
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )

In [53]:
import time

# Wait for the index to be ready
while not pc.describe_index(index_name).status['ready']:
    time.sleep(1)

index = pc.Index(index_name)

vectors = []
for _, row in df.iterrows():
    sanitized_filename = row["file"].encode('ascii', errors='ignore').decode('ascii')
    vectors.append({
        "id": sanitized_filename,  # must be string
        "values": row["image_embedding"],
        "metadata": {
            "title": row["title"],
            "artist": row["artist"],
            "style": row["style"],
            "genre": row["genre"]
        }
    })

index.upsert(
    vectors=vectors,
    namespace="ns1"
)

print(f"✅ Upserted {len(vectors)} vectors to Pinecone.")

✅ Upserted 30 vectors to Pinecone.


In [54]:
print(index.describe_index_stats())

{'dimension': 512,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'ns1': {'vector_count': 30}},
 'total_vector_count': 30,
 'vector_type': 'dense'}


In [55]:
# Load and preprocess your image
image_path = "data/raw/testing_images/test_8.png"
image = preprocess(Image.open(image_path)).unsqueeze(0).to(device)

# Get the image embedding
with torch.no_grad():
    image_embedding = model.encode_image(image).cpu().numpy().flatten().tolist()

# Search Pinecone index with the image embedding
query_response = index.query(
    namespace="ns1",
    vector=image_embedding,
    top_k=1,
    include_metadata=True
)

# View results
for match in query_response["matches"]:
    print(f"{match['id']} (score: {match['score']}) → {match['metadata']}")

2_Milking the Cow .jpg (score: 0.863982141) → {'artist': 'Yasuo Kuniyoshi', 'genre': 'genre painting', 'style': 'Expressionism', 'title': 'Milking the Cow '}


In [56]:
query_response

{'matches': [{'id': '2_Milking the Cow .jpg',
              'metadata': {'artist': 'Yasuo Kuniyoshi',
                           'genre': 'genre painting',
                           'style': 'Expressionism',
                           'title': 'Milking the Cow '},
              'score': 0.863982141,
              'values': []}],
 'namespace': 'ns1',
 'usage': {'read_units': 6}}

In [57]:
from scripts.get_embeddings import load_clip_model, get_image_embedding
from scripts.get_database import initialize_pinecone, create_pinecone_index, query_image, format_query_results
from dotenv import load_dotenv

load_dotenv()

PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
INDEX_NAME = "frame-finder-database"

model, preprocess, device = load_clip_model('RN101')
pc = initialize_pinecone(PINECONE_API_KEY)
index = create_pinecone_index(pc, INDEX_NAME)

image_embedding = get_image_embedding("data/raw/testing_images/test_8.png", model, preprocess, device)

query_response = query_image(image_embedding=image_embedding, index=index, top_k=1)

results = format_query_results(query_response)[0]

results 

Loading CLIP model: RN101 on cpu
Index frame-finder-database already exists


{'id': '2_Milking the Cow .jpg',
 'score': 0.863982141,
 'metadata': {'artist': 'Yasuo Kuniyoshi',
  'genre': 'genre painting',
  'style': 'Expressionism',
  'title': 'Milking the Cow '}}

In [58]:
results['metadata']

{'artist': 'Yasuo Kuniyoshi',
 'genre': 'genre painting',
 'style': 'Expressionism',
 'title': 'Milking the Cow '}

In [ ]:
from openai import OpenAI

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=OPENAI_API_KEY)

def get_art_explanation(results):
    prompt = f"""
You are an expert art historian.

Given the following painting metadata, generate a concise yet rich explanation including: 
1. The painting’s title and artist
2. What it represents
3. When and why it was painted (if known)
4. Any cultural or historical context

Metadata:
- Title: {results['metadata']['title']}
- Artist: {results['metadata']['artist']}
- Style: {results['metadata']['style']}
- Genre: {results['metadata']['genre']}

Return this as a structured paragraph.
"""

    response = client.responses.create(
        model="gpt-4o-mini",
        input = prompt
    )
    return response.output_text

op = get_art_explanation(results)
op